In [1]:
import boto3
import json, uuid, copy, datetime
import random, names, tabulate
import pandas as pd

In [2]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [3]:
# !./setuprds.sh
!./setupdynamo.sh

Creating the DynamoDB table that holds the client floats
{
    "StackId": "arn:aws:cloudformation:us-east-1:123456789:stack/client-float-table/3980ade8-82c7-47bf-8197-cbfcc17edd1c"
}
Adding in item for core, ZAR wholesale float


In [4]:
!./deploylambdas.sh

Deploying the float lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: Reconfiguring service lambda to use http://localhost:4574
Serverless: Reconfiguring service dynamodb to use http://localhost:4569
Serverless: Reconfiguring service kinesis to use http://localhost:4568
Serverless: Reconfiguring service route53 to use http://localhost:4580
Serverless: Reconfiguring service firehose to use http://localhost:4573
Serverless: Reconfiguring service stepfunctions to use http://localhost:4585
Serverless: Reconfiguring service es to use http://localhost:4578
Serverless: Reconfiguring serv

Serverless: Checking Stack create progress...
..
Serverless: Stack create finished...
Service Information
service: user-activity-api
stage: local
region: eu-west-1
stack: user-activity-api-local
resources: 12
api keys:
  None
endpoints:
  POST - https://748361A-Z255.execute-api.eu-west-1.amazonaws.com/local/save
functions:
  save: activity-save
  balance: activity-balance
layers:
  None
Serverless Enterprise: Run `serverless login` and deploy again to explore, monitor, secure your serverless project for free.
Deploying the migration lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless

In [5]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [6]:
# Setup the RDS roles
role_payload = { "type": "CREATE_ROLES", "credentials": {
        "account_api_worker": "pwd_for_account_api",
        "save_tx_api_worker": "pwd_for_transaction_api",
        "float_api_worker": "pwd_for_float_api",
        "auth_api_worker": "pwd_for_auth_api_worker"
    }
}
local_lambda.invoke(FunctionName='db-migration-local-migrate', InvocationType='RequestResponse', Payload=json.dumps(role_payload))

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '372',
   'access-control-allow-origin': '*',
   'server': 'Werkzeug/0.15.4 Python/3.6.8',
   'date': 'Wed, 03 Jul 2019 11:42:46 GMT'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'Payload': <botocore.response.StreamingBody at 0x7f28a0cfb208>}

In [ ]:
# Setup the RDS tables
local_lambda.invoke(FunctionName='db-migration-local-migrate', InvocationType='RequestResponse', Payload=json.dumps({'type': 'SETUP_TABLES'}))

In [7]:
def print_deployed_functions():
    function_list = local_lambda.list_functions()
    # print(function_list)
    print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

In [8]:
def generate_account(client_id = 'zar_client_co', float_id = 'zar_cash_float'):
    first_name = names.get_first_name()
    family_name = names.get_last_name()
    user_id = str(uuid.uuid4())
    return { 'clientId': client_id, 'floatId': float_id, 'ownerUserId': user_id, 'userFirstName': first_name, 'userFamilyName': family_name}

In [9]:
def decode_lambda_result(lambda_result):
    lambda_payload = lambda_result['Payload'].read()
    lambda_pload_decoded = lambda_payload.decode('utf-8')
    lambda_pload_object = json.loads(lambda_pload_decoded)
    return lambda_pload_object

In [10]:
def create_number_accounts(number_accounts = 1, client_id = 'zar_client_co'):
    account_dicts = [generate_account(client_id) for i in range(number_accounts)]
    persisted_accounts = []
    for account in account_dicts:
        lambda_result = local_lambda.invoke(FunctionName='account-create', InvocationType='RequestResponse', 
                                           Payload=json.dumps(account))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_account = copy.deepcopy(account)
        persisted_account['accountId'] = lambda_payload['accountId']
        persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Account with ID %s persisted at %s' % (persisted_account['accountId'], persisted_account['persistedTime']))
        persisted_accounts.append(persisted_account)
    
    return persisted_accounts
    

In [11]:
def generate_saving_transaction(account_id, ref_amount = 100, client_id = 'zar_client_co', float_id = 'zar_cash_primary'):
    current_time = datetime.datetime.now()
    saved_amount = round(random.random() * ref_amount * 10 * 10) # random proportion of ref amount, 
    test_saving_dict = { 
        'accountId': account_id, 
        'initiationTime': str(current_time), 
        'settlementTime': str(current_time), 
        'savedAmount': saved_amount, 
        'savedCurrency': 'ZAR', 
        'savedUnit': 'HUNDREDTH_CENT',
        'floatId': float_id,
        'clientId': client_id
    }
    return test_saving_dict

In [12]:
def seed_savings_for_accounts(accounts, tx_per_account = 1, base_amount = 100):
    transactions = []
    for i in range(tx_per_account):
        transactions.extend([generate_saving_transaction(account['accountId'], base_amount) for account in accounts])
#     print('transactions: ', transactions)
    tx_records = []
    for tx in transactions:
        lambda_result = local_lambda.invoke(FunctionName='activity-save', InvocationType='RequestResponse', 
                                           Payload=json.dumps(tx, default=str))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_tx = copy.deepcopy(tx)
        persisted_tx['transactionId'] = lambda_payload['transactionDetails'][0][0]['transaction_id']
        persisted_tx['currentBalance'] = lambda_payload['newBalance']['amount']
        # persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Tx for account %s persisted with new balance %s' % (tx['accountId'], persisted_tx['currentBalance']))
        tx_records.append(persisted_tx)
    
    return tx_records

In [13]:
def run_accrual_for_float(client_id = 'zar_client_co', float_id = 'zar_cash_primary', accrued_amount = 100):
    accrualDict = {
        'clientId': client_id,
        'floatId': float_id,
        'accrualAmount': accrued_amount,
        'currency': 'ZAR',
        'unit': 'HUNDREDTH_CENT',
        'backingEntityIdentifier': 'tx-id-backing'
    }
    lambda_result = local_lambda.invoke(FunctionName='float-accrue', InvocationType='RequestResponse', 
                                               Payload=json.dumps(accrualDict, default=str))
    lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
    return lambda_payload

In [14]:
print_deployed_functions()

Function list:  ['float-accrue', 'account-create', 'activity-save', 'activity-balance', 'db-migration-local-migrate']


In [15]:
accounts = create_number_accounts(number_accounts = 10)

Account with ID fe8973b4-f432-4a6b-9d95-8adc89a2131a persisted at 2019-07-03T11:44:56.111Z
Account with ID c028bd11-9d00-4e2e-9608-13950a41c0ae persisted at 2019-07-03T11:44:57.321Z
Account with ID 6b5c6580-5813-441e-a70d-3f7c1b989dda persisted at 2019-07-03T11:44:58.259Z
Account with ID 62e91b2b-5487-4e70-99bd-665f30cc0c58 persisted at 2019-07-03T11:44:58.960Z
Account with ID cbe6f008-2a68-43f9-b108-84633a3dd1fc persisted at 2019-07-03T11:44:59.621Z
Account with ID d95430eb-355a-47a6-b5db-1461c82f61a1 persisted at 2019-07-03T11:45:00.280Z
Account with ID b4870406-886e-47c4-a7a8-b9375df3a531 persisted at 2019-07-03T11:45:00.958Z
Account with ID 89867c26-0947-4a7a-a5c6-fc41ecd451a6 persisted at 2019-07-03T11:45:01.756Z
Account with ID 5063a2ec-6c4c-45aa-8aa3-8af7bd35e7ea persisted at 2019-07-03T11:45:02.702Z
Account with ID 1d50a956-2beb-4200-ba3e-5eee8233d5a1 persisted at 2019-07-03T11:45:03.414Z


In [16]:
account_info = [{ 'Name': account['userFirstName'] + ' ' + account['userFamilyName'], 
                'AccountId': account['accountId'] } for account in accounts[:100]]
print(tabulate.tabulate(account_info, headers = 'keys'))

Name             AccountId
---------------  ------------------------------------
Parker Wise      fe8973b4-f432-4a6b-9d95-8adc89a2131a
Bethany Scott    c028bd11-9d00-4e2e-9608-13950a41c0ae
Joseph Begley    6b5c6580-5813-441e-a70d-3f7c1b989dda
Evelyn Lowery    62e91b2b-5487-4e70-99bd-665f30cc0c58
Leona Young      cbe6f008-2a68-43f9-b108-84633a3dd1fc
Penelope Valeri  d95430eb-355a-47a6-b5db-1461c82f61a1
John Howard      b4870406-886e-47c4-a7a8-b9375df3a531
Carol Bradford   89867c26-0947-4a7a-a5c6-fc41ecd451a6
Cecil Allen      5063a2ec-6c4c-45aa-8aa3-8af7bd35e7ea
Teresa Gibson    1d50a956-2beb-4200-ba3e-5eee8233d5a1


In [17]:
single_tx = generate_saving_transaction(accounts[0]['accountId'], 100)
lambda_result = local_lambda.invoke(FunctionName='activity-save', Payload=json.dumps(single_tx, default=str))
result_decoded = decode_lambda_result(lambda_result)
print('Lambda result', result_decoded)

Lambda result {'body': '{"transactionDetails":[[{"transaction_id":"e3f3e552-4be9-46df-ada2-9776c769ba3c","creation_time":"2019-07-03T11:51:32.513Z"}],[{"transaction_id":"d53b789e-6104-4296-9129-0c51d9222633","creation_time":"2019-07-03T11:51:32.513Z"},{"transaction_id":"d53b789e-6104-4296-9129-0c51d9222633","creation_time":"2019-07-03T11:51:32.513Z"}]],"newBalance":{"amount":0,"unit":"HUNDREDTH_CENT"}}', 'statusCode': 200}


In [18]:
account_balances = seed_savings_for_accounts(accounts[:100], tx_per_account = 3, base_amount = 100 * 100 * 100)

Tx for account fe8973b4-f432-4a6b-9d95-8adc89a2131a persisted with new balance 6857
Tx for account c028bd11-9d00-4e2e-9608-13950a41c0ae persisted with new balance 0
Tx for account 6b5c6580-5813-441e-a70d-3f7c1b989dda persisted with new balance 0
Tx for account 62e91b2b-5487-4e70-99bd-665f30cc0c58 persisted with new balance 0
Tx for account cbe6f008-2a68-43f9-b108-84633a3dd1fc persisted with new balance 0
Tx for account d95430eb-355a-47a6-b5db-1461c82f61a1 persisted with new balance 0
Tx for account b4870406-886e-47c4-a7a8-b9375df3a531 persisted with new balance 0
Tx for account 89867c26-0947-4a7a-a5c6-fc41ecd451a6 persisted with new balance 0
Tx for account 5063a2ec-6c4c-45aa-8aa3-8af7bd35e7ea persisted with new balance 0
Tx for account 1d50a956-2beb-4200-ba3e-5eee8233d5a1 persisted with new balance 0
Tx for account fe8973b4-f432-4a6b-9d95-8adc89a2131a persisted with new balance 63103363
Tx for account c028bd11-9d00-4e2e-9608-13950a41c0ae persisted with new balance 39902817
Tx for acco

In [19]:
def merge_dataframes(accounts, account_balances):
    df = pd.DataFrame(account_balances)
    df = df[df['currentBalance'] == df.groupby('accountId')['currentBalance'].transform('max')]
    df = df[['accountId', 'currentBalance']]
    df['currentBalance'] = pd.to_numeric(df['currentBalance'])
    df['balanceRand'] = df['currentBalance']/10000
    account_name_df = pd.DataFrame(accounts)[['accountId', 'floatId', 'userFamilyName', 'userFirstName']]
    account_name_df['Holder Name'] = account_name_df['userFirstName'] + ' ' + account_name_df['userFamilyName']
    account_name_df.set_index('accountId')
    df.set_index('accountId')
    merged_df = pd.merge(account_name_df, df)[['accountId', 'floatId', 'Holder Name', 'balanceRand']]
    return merged_df

In [20]:
merged_df = merge_dataframes(accounts, account_balances)

In [21]:
merged_df

,accountId,floatId,Holder Name,balanceRand
0,fe8973b4-f432-4a6b-9d95-8adc89a2131a,zar_cash_float,Parker Wise,13148.1991
1,c028bd11-9d00-4e2e-9608-13950a41c0ae,zar_cash_float,Bethany Scott,10833.8015
2,6b5c6580-5813-441e-a70d-3f7c1b989dda,zar_cash_float,Joseph Begley,13548.2138
3,62e91b2b-5487-4e70-99bd-665f30cc0c58,zar_cash_float,Evelyn Lowery,14099.6812
4,cbe6f008-2a68-43f9-b108-84633a3dd1fc,zar_cash_float,Leona Young,10674.7788
5,d95430eb-355a-47a6-b5db-1461c82f61a1,zar_cash_float,Penelope Valeri,14468.9870
6,b4870406-886e-47c4-a7a8-b9375df3a531,zar_cash_float,John Howard,7405.1445
7,89867c26-0947-4a7a-a5c6-fc41ecd451a6,zar_cash_float,Carol Bradford,5271.0045
8,5063a2ec-6c4c-45aa-8aa3-8af7bd35e7ea,zar_cash_float,Cecil Allen,11528.1001
9,1d50a956-2beb-4200-ba3e-5eee8233d5a1,zar_cash_float,Teresa Gibson,11208.9176


In [22]:
amount_to_accrue = round(random.random() * 100 * 100 * 100)
print('Instructing accrual of : R', amount_to_accrue / (100 * 100))
accrual_result = run_accrual_for_float(accrued_amount = amount_to_accrue)
allocation_details = json.loads(accrual_result['userAllocationTransactions']['body'])
account_transaction_list = allocation_details['allocationRecords']['accountTxIds']
# print(tabulate.tabulate(account_transaction_list, headers = 'keys'))

Instructing accrual of : R 14.419


In [23]:
# print(accrual_result)
account_transactions = allocation_details['allocationRecords']['accountTxIds']
number_of_float_transactions = len(allocation_details['allocationRecords']['floatTxIds'])
number_of_account_transactions = len(account_transactions)
print('%d float transactions, %d account transactions' % (number_of_float_transactions, number_of_account_transactions))
sum_of_accounts = sum([transaction['amount'] for transaction in account_transactions])
print('Sum of account additions: ', sum_of_accounts)
bonus_allocation = accrual_result['entityAllocations']['bonusShare']
company_allocation = accrual_result['entityAllocations']['clientShare']
print('Bonus allocation: %d, and client co allocation: %d' % (bonus_allocation, company_allocation))
total_allocations = sum_of_accounts + bonus_allocation + company_allocation
print('Of accrual %d, allocations %d' % (amount_to_accrue, total_allocations))
rounding_allocated_to_bonus = isinstance(allocation_details['bonusAllocation'], list)
print('Was there a rounding allocation? : ', rounding_allocated_to_bonus)
print('Rounding allocation record: ', allocation_details['bonusAllocation'])

21 float transactions, 21 account transactions
Sum of account additions:  115352
Bonus allocation: 14419, and client co allocation: 14419
Of accrual 144190, allocations 144190
Was there a rounding allocation? :  False
Rounding allocation record:  {}


In [ ]:
balance_payload = { 
    'accountId': accounts[0]['accountId'], 
    'clientId': 'zar_client_co', 
    'floatId': 'zar_cash_primary',
    'currency': 'ZAR',
    'timezone': 'Africa/Johannesburg'
}
balance_projection = local_lambda.invoke(FunctionName='activity-balance', Payload=json.dumps(balance_payload))
projection_decoded = decode_lambda_result(balance_projection)
print(projection_decoded)

In [24]:
def fetch_account_balance(accountId, clientId = 'zar_client_co', floatId = 'zar_cash_primary', currency = 'ZAR', timezone = 'Africa/Johannesburg'):
    balance_payload = { 
        'accountId': accountId, 
        'clientId': 'zar_client_co', 
        'floatId': 'zar_cash_primary',
        'currency': 'ZAR',
        'timezone': 'Africa/Johannesburg'
    }
    balance_projection = local_lambda.invoke(FunctionName='activity-balance', Payload=json.dumps(balance_payload))
    projection_decoded = decode_lambda_result(balance_projection)
    return projection_decoded

In [29]:
def fetch_account_balances(accounts):
#     balance_dicts = [fetch_account_balance(account['accountId']) for account in accounts]
    balance_dicts = []
    for account in accounts:
#         print('account: ', account)
        balance_dict = fetch_account_balance(account['accountId'])
        balance_dict['accountId'] = account['accountId']
        projected_dict = {
            'accountId': balance_dict['accountId'],
            'currentBalanceZAR': balance_dict['currentBalance']['amount'] / 10000,
            'balanceEndOfDayZAR': balance_dict['balanceEndOfToday']['amount'] / 10000,
            'balanceNextDayZAR': balance_dict['balanceSubsequentDays'][0]['amount'] / 10000
        }
        balance_dicts.append(projected_dict)
    
    return balance_dicts

In [30]:
balances_projections = fetch_account_balances(accounts)

In [39]:
balance_df = pd.DataFrame(balances_projections)
balance_df = balance_df[['accountId', 'currentBalanceZAR', 'balanceEndOfDayZAR', 'balanceNextDayZAR']]
balance_df

,accountId,currentBalanceZAR,balanceEndOfDayZAR,balanceNextDayZAR
0,fe8973b4-f432-4a6b-9d95-8adc89a2131a,22587.4507,22590.4830,22593.5157
1,c028bd11-9d00-4e2e-9608-13950a41c0ae,15951.9645,15954.1060,15956.2478
2,6b5c6580-5813-441e-a70d-3f7c1b989dda,15627.4378,15629.5357,15631.6339
3,62e91b2b-5487-4e70-99bd-665f30cc0c58,24013.7314,24016.9552,24020.1794
4,cbe6f008-2a68-43f9-b108-84633a3dd1fc,15601.2192,15603.3136,15605.4083
5,d95430eb-355a-47a6-b5db-1461c82f61a1,23143.1100,23146.2169,23149.3242
6,b4870406-886e-47c4-a7a8-b9375df3a531,9569.4069,9570.6916,9571.9764
7,89867c26-0947-4a7a-a5c6-fc41ecd451a6,9468.1975,9469.4686,9470.7398
8,5063a2ec-6c4c-45aa-8aa3-8af7bd35e7ea,14922.6866,14924.6899,14926.6935
9,1d50a956-2beb-4200-ba3e-5eee8233d5a1,18741.2596,18743.7755,18746.2918
